## Neste código construiremos 3 redes neurais para aproximar derivadas.
## Nos meus testes os resultados foram positivos, mas não posso garantir a consistência dos resultados devido a aleatoriedade do conjunto de dados criado.

In [1]:
import tensorflow as tf
import numpy as np

float_pres='float64'

In [126]:
# Criando conjunto de dados

data_x_list=[]
data_y_list=[]
pi=np.pi

for i in range(5000):
    Δx = 0.01                                 # Distância espacial dos pontos na malha utilizada
    x = tf.range(-2, 2, Δx, dtype=float_pres) # Gerando a malha de pontos no espaço unidimensional
    
    # Gerando uma condição inicial aleatória
    #------------------------------------------------------------------------------------------------------------------
#     k1 = tf.random.uniform([1], 0, 20, dtype='int32')   # Amostrando uma frequência aleatória para a função seno
#     k1 = tf.cast(k1, dtype=float_pres)                  # Mudando o tipo do tensor
#     k2 = tf.random.uniform([1], 0, 20, dtype='int32')   # Amostrando uma frequência aleatória para a função seno
#     k2 = tf.cast(k2, dtype=float_pres)                  # Mudando o tipo do tensor
#     a  = tf.random.uniform([1], 0, 1, dtype=float_pres) # Amostrando um peso aleatória para ponderar as funções seno
#     b  = tf.random.uniform([1], 0, 2, dtype=float_pres) # Amostrando um modificador de amplitude aleatório
    #------------------------------------------------------------------------------------------------------------------
    
    # Valor da função
#     u1 =     a * tf.expand_dims(tf.math.sin(k1*pi*x), axis=0) # Gerando pontos de acordo com a primeira função seno
#     u2 = (1-a) * tf.expand_dims(tf.math.sin(k2*pi*x), axis=0) # Gerando pontos de acordo com a segunda função seno
    
    # Valor da derivada
#     du1= a*k1*pi*tf.expand_dims(tf.math.cos(k1*pi*x), axis=0)
#     du2= (1-a)*k2*pi*tf.expand_dims(tf.math.cos(k2*pi*x), axis=0)
    
#     u = b*(u1+u2) 
#     du= b*(du1+du2)
    
#     a = tf.random.uniform([1], -3, 3, dtype=float_pres)
#     b = tf.random.uniform([1], -3, 3, dtype=float_pres)
    
#     u  = tf.expand_dims(a*x + b, axis=0)
#     du = tf.expand_dims(0*x + a, axis=0)
    
#     data_x_list.append(u)
#     data_y_list.append(du)
    
    a = tf.random.uniform([1], -3, 3, dtype=float_pres)
    b = tf.random.uniform([1], -3, 3, dtype=float_pres)
    c = tf.random.uniform([1], -1, 1, dtype=float_pres)
    
    u  = tf.expand_dims(  a*x**2 + b*x + c, axis=0)
    du = tf.expand_dims(2*a*x    + b      , axis=0)
    
    data_x_list.append(u)
    data_y_list.append(du)
    
    a = tf.random.uniform([1], -2, 2, dtype=float_pres)
    b = tf.random.uniform([1], -3, 3, dtype=float_pres)
    c = tf.random.uniform([1], -3, 3, dtype=float_pres)
    d = tf.random.uniform([1], -3, 3, dtype=float_pres)
    
    u  = tf.expand_dims(  a*x**3 +   b*x**2 + c*x + d, axis=0)
    du = tf.expand_dims(3*a*x**2 + 2*b*x    + c      , axis=0)
    
    data_x_list.append(u)
    data_y_list.append(du)

data_x = tf.concat(data_x_list,axis=0)
data_y = tf.concat(data_y_list,axis=0)

train_x = data_x[:40000]
train_y = data_y[:40000]
test_x  = data_x[-40000:]
test_y  = data_y[-40000:]

In [158]:
# Criando um modelo sequencial
model_foward=tf.keras.models.Sequential()

# Adicionando camadas
model_foward.add(tf.keras.layers.Reshape([400,1])) # A camada de convolução exige uma dimensão extra para a "cor" da imagem,
                                            # por isso o reshape para transformar o tensor de tamanho n x 400 em um tensor de tamanho n x 400 x 1
model_foward.add(tf.keras.layers.Conv1D(filters=1,activation='linear', kernel_size=4, use_bias=False,dtype=float_pres)) # Camada de convolução com ativação linear, após a convolução, a dimensão do vetor será n x 399.
                                                                                                       # Lembrando que, se temos um input com dimensão n x s x r e uma convolução com filtro a x b (a é o tamanho do kernel e b é a quantidade de filtros) 
                                                                                                       # Então o output da convolução será: n x (s-a+1) x b
                                                                                                       # Dada a natureza do problema, sabemos que não é necessário um bias (use_bias=False), então removemos ele para evitar overfitting.
model_foward.add(tf.keras.layers.Flatten()) # Esta camada remove a dimensão extra, transformando um tensor de tamanho n x 399 x 1 em um tensor de tamanho n x 399


optimizer = tf.keras.optimizers.Adam(learning_rate=10**-1, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
model_foward.compile(loss='mean_squared_error',optimizer=optimizer)

In [159]:
from IPython.display import clear_output
class ClearOutput(tf.keras.callbacks.Callback):
    def on_epoch_end(self, *logs):
        clear_output()


In [170]:
history = model_foward.fit(train_x, train_y[:,1:-2], # Como estamos calculando a derivada "pra frente", devemos remover a última derivada
                           batch_size=512,
                           epochs=30,
                           callbacks = [ClearOutput()],
                           validation_split=0.2)
test_scores = model_foward.evaluate(test_x, test_y[:,1:-2], verbose=1)
print('Test loss:', test_scores)

625/625 [==============================] - 1s 1ms/step - loss: 0.0568
Test loss: 0.056810710579156876


In [171]:
# Pegando os pesos da segunda camada (a de convolução) da rede e multiplicando por Δx para ver se o valor está correto (o ideal seria -1 e 1).
model_foward.layers[1].weights[0]*Δx

<tf.Tensor: shape=(4, 1, 1), dtype=float64, numpy=
array([[[-0.29447868]],

       [[-0.11034934]],

       [[ 0.10817446]],

       [[ 0.29608895]]])>

In [166]:
k1 = 10

u  =       tf.expand_dims(tf.math.sin(k1*pi*x), axis=0) # Gerando pontos de acordo com a primeira função seno
du = k1*pi*tf.expand_dims(tf.math.cos(k1*pi*x), axis=0) # Gerando pontos de acordo com a primeira função coseno

# a = model_foward.layers[1].weights[0][0]*Δx
# b = model_foward.layers[1].weights[0][1]*Δx
# c = model_foward.layers[1].weights[0][2]*Δx
# d = model_foward.layers[1].weights[0][3]*Δx

model_foward.evaluate(u, du[:,1:-2])

1/1 [==============================] - 0s 17ms/step - loss: 15.2933


15.29328441619873

In [172]:
# Coeficientes exatos
a = -3/10
b = -1/10
c =  1/10
d =  3/10

print("Coeficientes Analíticos: ",tf.math.reduce_mean(((a*test_x[:,:-3]+b*test_x[:,1:-2]+c*test_x[:,2:-1]+d*test_x[:,3:])/Δx-test_y[:,1:-2])**2))

# Outros coeficientes
a = model_foward.layers[1].weights[0][0]*Δx
b = model_foward.layers[1].weights[0][1]*Δx
c = model_foward.layers[1].weights[0][2]*Δx
d = model_foward.layers[1].weights[0][3]*Δx

print("Coeficientes Apreendidos:",tf.math.reduce_mean(((a*test_x[:,:-3]+b*test_x[:,1:-2]+c*test_x[:,2:-1]+d*test_x[:,3:])/Δx-test_y[:,1:-2])**2))


Coeficientes Analíticos:  tf.Tensor(0.001083501900011149, shape=(), dtype=float64)
Coeficientes Apreendidos: tf.Tensor(0.05681070189231972, shape=(), dtype=float64)


In [ ]:
# Criando um modelo sequencial
model_backward=tf.keras.models.Sequential()

# Adicionando camadas
model_backward.add(tf.keras.layers.Reshape([400,1])) # A camada de convolução exige uma dimensão extra para a "cor" da imagem,
                                            # por isso o reshape para transformar o tensor de tamanho n x 400 em um tensor de tamanho n x 400 x 1
model_backward.add(tf.keras.layers.Conv1D(filters=1,activation='linear', kernel_size=2, use_bias=False)) # Camada de convolução com ativação linear, após a convolução, a dimensão do vetor será n x 399.
                                                                                                         # Lembrando que, se temos um input com dimensão n x s x r e uma convolução com filtro a x b (a é o tamanho do kernel e b é a quantidade de filtros) 
                                                                                                         # Então o output da convolução será: n x (s-a+1) x b
model_backward.add(tf.keras.layers.Flatten()) # Esta camada remove a dimensão extra, transformando um tensor de tamanho n x 399 x 1 em um tensor de tamanho n x 399


optimizer = tf.keras.optimizers.Adam(learning_rate=10**-1, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
model_backward.compile(loss='mean_squared_error',optimizer=optimizer,metrics=['mean_squared_error'])

In [ ]:
history = model_backward.fit(train_x, train_y[:,1:], # Como estamos calculando a derivada "pra trás", devemos remover a primeira derivada
                    batch_size=512,
                    epochs=30,
                    validation_split=0.2)
test_scores = model_backward.evaluate(test_x, test_y[:,1:], verbose=1)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

In [ ]:
# Pegando os pesos da segunda camada (a de convolução) da rede e multiplicando por Δx para ver se o valor está correto (o ideal seria -1 e 1).
model_backward.layers[1].weights[0]*Δx

In [ ]:
# Criando um modelo sequencial
model_middle=tf.keras.models.Sequential()

# Adicionando camadas
model_middle.add(tf.keras.layers.Reshape([400,1])) # A camada de convolução exige uma dimensão extra para a "cor" da imagem,
                                            # por isso o reshape para transformar o tensor de tamanho n x 400 em um tensor de tamanho n x 400 x 1
# Observe que, para a derivada centrada, o kernel_size deve ser 3.
model_middle.add(tf.keras.layers.Conv1D(filters=1,activation='linear', kernel_size=3, use_bias=False)) # Camada de convolução com ativação linear, após a convolução, a dimensão do vetor será n x 398.
                                                                                                       # Lembrando que, se temos um input com dimensão n x s x r e uma convolução com filtro a x b (a é o tamanho do kernel e b é a quantidade de filtros) 
                                                                                                       # Então o output da convolução será: n x (s-a+1) x b
model_middle.add(tf.keras.layers.Flatten()) # Esta camada remove a dimensão extra, transformando um tensor de tamanho n x 399 x 1 em um tensor de tamanho n x 399


optimizer = tf.keras.optimizers.Adam(learning_rate=10**-1, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
model_middle.compile(loss='mean_squared_error',optimizer=optimizer,metrics=['mean_squared_error'])

In [ ]:
history = model_middle.fit(train_x, train_y[:,1:-1], # Como estamos calculando a derivada centrada, devemos remover a primeira e a última derivada
                    batch_size=512,
                    epochs=30,
                    validation_split=0.2)
test_scores = model_middle.evaluate(test_x, test_y[:,1:-1], verbose=1)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

In [ ]:
# Pegando os pesos da segunda camada (a de convolução) da rede e multiplicando por Δx para ver se o valor está correto (o ideal seria -1 e 1).
model_middle.layers[1].weights[0]*Δx